In [1]:
%matplotlib inline

import os
import warnings

import numpy as np
import matplotlib.pyplot as plt
import cv2

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications import VGG16

from sklearn.model_selection import train_test_split
# Note: Using Tensorflow 2

## Data Processing and Normalization

In [3]:
# Load data
input_size = 160

x_data = []
y_data = []
path_data = './data/'

# Đọc ảnh thư mục trong sáng, resize ảnh cho phù hợp đầu vào mạng, chuẩn hóa, thêm vào list chứa dữ liệu
for i_name in os.listdir(path_data + '0'):
    im = cv2.imread(path_data + '0/' + i_name)
    im_resize = cv2.resize(im, (input_size, input_size))
    im_resize = im_resize/255
    x_data.append(im_resize)
    y_data.append([0])
# Đọc ảnh thư mục gợi cảm, resize ảnh cho phù hợp đầu vào mạng, chuẩn hóa, thêm vào list chứa dữ liệu
for i_name in os.listdir(path_data + '1'):
    im = cv2.imread(path_data + '1/' + i_name)
    im_resize = cv2.resize(im, (input_size, input_size))
    im_resize = im_resize/255
    x_data.append(im_resize)
    y_data.append([1])
    
print('Finished loading')

In [4]:
# Images to array
x_data = np.array(x_data)
y_data = np.array(y_data)
np.save('./data/x_data.npy', x_data)
np.save('./data/y_data.npy', y_data)
print(x_data.shape)
print(y_data.shape)

(704, 160, 160, 3)
(704, 1)


In [10]:
# Load data
x_data = np.load('./data/x_data.npy')
y_data = np.load('./data/y_data.npy')

In [11]:
# Chia dữ liệu ra làm 2 phần: 1 phần để train, 1 phần để đánh giá
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=0)
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(633, 160, 160, 3)
(633, 1)
(71, 160, 160, 3)
(71, 1)


## 1. Build a model CNN for Classification

In [12]:
# Define model
input_size = 160

input_layer = Input(shape=(input_size, input_size, 3))
conv_1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
maxpool_1 = MaxPooling2D((2, 2))(conv_1)
conv_2 = Conv2D(64, (3, 3), activation='relu', padding='same')(maxpool_1)
maxpool_2 = MaxPooling2D((2, 2))(conv_2)
conv_3 = Conv2D(128, (3, 3), activation='relu', padding='same')(maxpool_2)
maxpool_3 = MaxPooling2D((2, 2))(conv_3)
conv_4 = Conv2D(128, (3, 3), activation='relu', padding='same')(maxpool_3)
maxpool_4 = MaxPooling2D((2, 2))(conv_4)
flatten = Flatten()(maxpool_4)
dropout = Dropout(0.5)(flatten)
fully_1 = Dense(512, activation='relu')(dropout)
fully_2 = Dense(256, activation='relu')(fully_1)
output_layer = Dense(1, activation='sigmoid')(fully_2)

model = Model(input_layer, output_layer)

#opt = SGD(lr=0.001, momentum=0.9) # SGD Optimizer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 160, 160, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 80, 80, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 80, 80, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 40, 40, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 40, 40, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 20, 20, 128)       0   

In [24]:
# Training model
history = model.fit(x_train, y_train, verbose=1, epochs=3, validation_data=(x_val, y_val), shuffle=True, batch_size=32)

Train on 633 samples, validate on 71 samples
Epoch 1/3
633/633 [==============================] - 17s 26ms/sample - loss: 0.4935 - accuracy: 0.7646 - val_loss: 0.6016 - val_accuracy: 0.6901
Epoch 2/3
633/633 [==============================] - 17s 26ms/sample - loss: 0.4791 - accuracy: 0.7472 - val_loss: 0.5842 - val_accuracy: 0.7324
Epoch 3/3
633/633 [==============================] - 17s 26ms/sample - loss: 0.4336 - accuracy: 0.8025 - val_loss: 0.6351 - val_accuracy: 0.6901


In [27]:
# Evaluate model
loss_val, acc_val = model.evaluate(x_val, y_val)
print("accuracy = ",acc_val)

71/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [28]:
# Save model
model.save('./models/model.h5')
print("DONE")

DONE


## 2. Using VGG16 pre-trained for transfer learning

In [2]:
# Load model VGG16
base_model = VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

Model loaded.


In [15]:
# Load data
#input_size = 244 # Original VGG16 train on Imagenet
input_size = 150

x_data = []
y_data = []
path_data = './data/'

# Đọc ảnh thư mục trong sáng, resize ảnh cho phù hợp đầu vào mạng, chuẩn hóa, thêm vào list chứa dữ liệu
for i_name in os.listdir(path_data + '0'):
    im = cv2.imread(path_data + '0/' + i_name)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im_resize = cv2.resize(im, (input_size, input_size)) * 1.0
    im_resize -= [123.68, 116.779, 103.939]
    x_data.append(im_resize)
    y_data.append([0])
# Đọc ảnh thư mục gợi cảm, resize ảnh cho phù hợp đầu vào mạng, chuẩn hóa, thêm vào list chứa dữ liệu
for i_name in os.listdir(path_data + '1'):
    im = cv2.imread(path_data + '1/' + i_name)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im_resize = cv2.resize(im, (input_size, input_size)) * 1.0
    im_resize -= [123.68, 116.779, 103.939]
    x_data.append(im_resize)
    y_data.append([1])
    
print('Finished loading')

Finished loading


In [16]:
# Images to array
x_data = np.array(x_data)
y_data = np.array(y_data)
#np.save('./data/x_data.npy', x_data)
#np.save('./data/y_data.npy', y_data)
print(x_data.shape)
print(y_data.shape)

(704, 150, 150, 3)
(704, 1)


In [50]:
# Chia dữ liệu ra làm 2 phần: 1 phần để train, 1 phần để đánh giá
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=0)
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(633, 150, 150, 3)
(633, 1)
(71, 150, 150, 3)
(71, 1)


In [58]:
np.save('./data/x_train.npy', x_train)
np.save('./data/y_train.npy', y_train)
np.save('./data/x_val.npy', x_val)
np.save('./data/y_val.npy', y_val)
print('label saved')

label saved


In [24]:
# Feature Extract
Feature_x_train = base_model.predict(x_train)
np.save('./data/Feature_x_train.npy', Feature_x_train)
print(Feature_x_train.shape)
Feature_x_val = base_model.predict(x_val)
np.save('./data/Feature_x_val.npy', Feature_x_val)
print(Feature_x_val.shape)

(633, 4, 4, 512)
(71, 4, 4, 512)


In [26]:
# Load data
Feature_x_train = np.load('./data/Feature_x_train.npy')
Feature_x_val = np.load('./data/Feature_x_val.npy')
y_train = np.load('./data/y_train.npy')
y_val = np.load('./data/y_val.npy')
print(Feature_x_train.shape)
print(y_train.shape)
print(Feature_x_val.shape)
print(y_val.shape)

(633, 4, 4, 512)
(633, 1)
(71, 4, 4, 512)
(71, 1)


In [27]:
# Define model classification
input_layer = Input(shape=Feature_x_train.shape[1:])
flat_1 = Flatten()(input_layer)
fully_1 = Dense(256, activation='relu')(flat_1)
drop_1 = Dropout(0.5)(fully_1)
output_layer = Dense(1, activation='sigmoid')(drop_1)

fc_model = Model(input_layer, output_layer)

fc_model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
fc_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 4, 4, 512)]       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 2,097,665
Trainable params: 2,097,665
Non-trainable params: 0
_________________________________________________________________


In [43]:
#Training model
fc_model.fit(Feature_x_train, y_train, epochs=1, batch_size=16, validation_data=(Feature_x_val, y_val))

Train on 633 samples, validate on 71 samples
633/633 [==============================] - 1s 2ms/sample - loss: 1.5342e-08 - accuracy: 1.0000 - val_loss: 5.7076 - val_accuracy: 0.9014


In [45]:
# Save model
fc_model.save_weights('./models/fc_model.h5')
print("model saved")

model saved


### Final model

In [47]:
# dimensions of our images.
img_width, img_height = 150, 150

# Load VGG16 model
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
print('Model loaded.')

Model loaded.


In [48]:
# build a classifier model to put on top of the convolutional model
input_layer = Input(shape=vgg16.output_shape[1:])
flat_1 = Flatten()(input_layer)
fully_1 = Dense(256, activation='relu')(flat_1)
drop_1 = Dropout(0.5)(fully_1)
output_layer = Dense(1, activation='sigmoid')(drop_1)

top_model = Model(input_layer, output_layer)

top_model.load_weights('./models/fc_model.h5')

print("Builded model")

Builded model


In [49]:
# Final model
final_model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))
final_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0   

In [53]:
# set the first 15 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in final_model.layers[:15]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
final_model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.00001, momentum=0.9), metrics=['accuracy'])

In [54]:
# Test final model
final_model.evaluate(x_val, y_val)

71/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[5.707606483513201, 0.90140843]

In [57]:
# Save final model
final_model.save('./models/final_model.h5')
print("model saved")

model saved
